## Ensuring Consistency in Multi-source Data Integration

**Description**: Validate the integration of two datasets `products_A.csv` and `products_B.csv` . Ensure consistency in product "category" information.

In [ ]:

import pandas as pd
import numpy as np

# --- 1. Load Data ---
try:
    df_A = pd.read_csv("products_A.csv")
    print("products_A.csv loaded successfully.")
except FileNotFoundError:
    print("Error: 'products_A.csv' not found. Creating a dummy DataFrame.")
    df_A = pd.DataFrame({
        'product_id': ['P001', 'P002', 'P003', 'P004', 'P005'],
        'product_name': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
        'category': ['Electronics', 'Peripherals', 'Peripherals', 'Electronics', 'Peripherals'],
        'price_A': [1200, 25, 75, 300, 50]
    })

try:
    df_B = pd.read_csv("products_B.csv")
    print("products_B.csv loaded successfully.")
except FileNotFoundError:
    print("Error: 'products_B.csv' not found. Creating a dummy DataFrame.")
    df_B = pd.DataFrame({
        'product_id': ['P001', 'P002', 'P003', 'P004', 'P006'],
        'product_name': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Printer'],
        'category': ['Electronics', 'Peripheral', 'Peripherals', 'Computers', 'Peripherals'], # Note: 'Peripheral' vs 'Peripherals', 'Computers' vs 'Electronics'
        'stock_B': [50, 100, 75, 30, 20]
    })

print("\n--- DataFrames Loaded ---")
print("products_A.head():")
print(df_A.head())
print("\nproducts_B.head():")
print(df_B.head())

# --- 2. Identify Common Keys ---
# Assuming 'product_id' is the primary key for joining.
common_key = 'product_id'

# --- 3. Merge Data ---
# Using an outer merge to include all products from both datasets.
# We'll keep both 'category' columns to compare them.
merged_df = pd.merge(df_A, df_B, on=common_key, how='outer', suffixes=('_A', '_B'))

print("\n--- Merged DataFrame (Partial View) ---")
print(merged_df.head())

# --- 4. Compare 'category' Information and 5. Identify Inconsistencies ---
# Convert categories to a consistent case (e.g., lowercase) and strip whitespace
merged_df['category_A_clean'] = merged_df['category_A'].str.lower().str.strip()
merged_df['category_B_clean'] = merged_df['category_B'].str.lower().str.strip()

# Identify inconsistencies where both categories exist but are different
inconsistent_categories = merged_df[
    (merged_df['category_A_clean'].notna()) &
    (merged_df['category_B_clean'].notna()) &
    (merged_df['category_A_clean'] != merged_df['category_B_clean'])
]

print("\n--- Products with Inconsistent Categories ---")
if not inconsistent_categories.empty:
    print(inconsistent_categories[[common_key, 'product_name', 'category_A', 'category_B']])
else:
    print("No direct category inconsistencies found for common products.")

# Also check for products present in one dataset but not the other,
# which might imply missing category information or new products.
products_only_A = merged_df[merged_df['category_B'].isna() & merged_df['category_A'].notna()]
products_only_B = merged_df[merged_df['category_A'].isna() & merged_df['category_B'].notna()]

if not products_only_A.empty:
    print("\n--- Products found only in products_A (missing category in B) ---")
    print(products_only_A[[common_key, 'product_name', 'category_A']])

if not products_only_B.empty:
    print("\n--- Products found only in products_B (missing category in A) ---")
    print(products_only_B[[common_key, 'product_name', 'category_B']])

# --- 6. Analyze and Resolve (Example of a simple resolution strategy) ---
# For demonstration, let's create a 'final_category' column.
# Resolution strategy:
# 1. If categories are consistent, use that category.
# 2. If inconsistent, prioritize category from A (or B, or flag for review).
# 3. If category is only in A or B, use that category.

merged_df['final_category'] = merged_df['category_A_clean']

# If category_A_clean is NaN, but category_B_clean is not NaN, use category_B_clean
merged_df.loc[merged_df['category_A_clean'].isna(), 'final_category'] = merged_df['category_B_clean']

# For inconsistent cases, we need a rule. Let's say we prioritize 'category_A'
# You would define your business logic here.
# For this example, if there's an inconsistency, we'll keep the category from A.
# If you wanted to flag them, you could add a 'consistency_flag' column.
merged_df['consistency_flag'] = 'Consistent'
merged_df.loc[
    (merged_df['category_A_clean'].notna()) &
    (merged_df['category_B_clean'].notna()) &
    (merged_df['category_A_clean'] != merged_df['category_B_clean']),
    'consistency_flag'
] = 'Inconsistent - Needs Review'

print("\n--- Merged DataFrame with Final Category and Consistency Flag ---")
print(merged_df[[common_key, 'product_name', 'category_A', 'category_B', 'final_category', 'consistency_flag']].head(10))

print("\nSummary of Consistency Flags:")
print(merged_df['consistency_flag'].value_counts())

# You can now save this reconciled DataFrame
# merged_df.to_csv("integrated_products_with_consistent_categories.csv", index=False)

Error: 'products_A.csv' not found. Creating a dummy DataFrame.
Error: 'products_B.csv' not found. Creating a dummy DataFrame.

--- DataFrames Loaded ---
products_A.head():
  product_id product_name     category  price_A
0       P001       Laptop  Electronics     1200
1       P002        Mouse  Peripherals       25
2       P003     Keyboard  Peripherals       75
3       P004      Monitor  Electronics      300
4       P005       Webcam  Peripherals       50

products_B.head():
  product_id product_name     category  stock_B
0       P001       Laptop  Electronics       50
1       P002        Mouse   Peripheral      100
2       P003     Keyboard  Peripherals       75
3       P004      Monitor    Computers       30
4       P006      Printer  Peripherals       20

--- Merged DataFrame (Partial View) ---
  product_id product_name_A   category_A  price_A product_name_B   category_B  \
0       P001         Laptop  Electronics   1200.0         Laptop  Electronics   
1       P002          Mouse  

KeyError: "['product_name'] not in index"